In [1]:
# VAE architecture


import torch
from models.base import BaseVAE
from torch import nn
from torch.nn import functional as F
from models.types_ import *


class ResidualBlockEncoder(nn.Module):
    
    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 stride: int,
                 **kwargs) -> None:
        super(ResidualBlockEncoder, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.stride = stride
        
        self.conv1 = nn.Conv2d(in_channels, out_channels,
                              kernel_size= 3, stride= stride, padding  = 1)
        self.conv2 = nn.Conv2d(in_channels, out_channels,
                              kernel_size= 3, stride= stride, padding  = 1)
        self.norm = nn.BatchNorm2d(in_channels)
        self.relu = nn.LeakyReLU()
        
    def forward(self, x):
        x = self.norm(x)
        if self.stride != 1 or self.in_channels != self.out_channels:
            temp = self.conv2(x)
        else: temp = 1*x
        x = self.relu(self.conv1(x))
        x = x + temp
        return x
    
    
class ResidualBlockDecoder(nn.Module):
    
    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 stride: int,
                 **kwargs) -> None:
        super(ResidualBlockDecoder, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.stride = stride
        
        self.conv1 = nn.ConvTranspose2d(in_channels, 
                                        out_channels,
                                        kernel_size= 3, 
                                        stride= stride, 
                                        padding  = 1,
                                        output_padding=stride-1)
        
        self.conv2 = nn.ConvTranspose2d(in_channels, 
                                        out_channels,
                                        kernel_size= 3, 
                                        stride= stride, 
                                        padding  = 1,
                                        output_padding=stride-1)
        self.norm = nn.BatchNorm2d(in_channels)
        self.relu = nn.LeakyReLU()
        
    def forward(self, x):
        x = self.norm(x)
        if self.stride != 1 or self.in_channels != self.out_channels:
            temp = self.conv2(x)
        else: temp = 1*x
        x = self.relu(self.conv1(x))
        x = x + temp
        return x

        
class VAE(BaseVAE):

    def __init__(self,
                 in_channels: int,
                 in_size: int, 
                 latent_dim: int,
                 hidden_dims: List = None,
                 **kwargs) -> None:
        super(VAE, self).__init__()

        self.latent_dim = latent_dim
        self.nb_last_channels = hidden_dims[-1]
        self.out_channels = in_channels
        

        modules = []
        if hidden_dims is None:
            hidden_dims = [32, 64, 128, 256, 512]
            div = 2**5  # Model reduces size by this factor           
        else:
            div = 2**len(hidden_dims)   # Model reduces size by this factor   
            
        # Make sure input dimension is usable     
        if in_size%div==0:
            self.smallest_size = int(in_size/div)
        else: raise ValueError('Input size not compatible with number of hidden layers.')

            

        # Build Encoder
        for h_dim in hidden_dims:
            modules.append(
                nn.Sequential(
                    ResidualBlockEncoder(in_channels=in_channels, out_channels=in_channels, stride= 1),
                    ResidualBlockEncoder(in_channels=in_channels, out_channels=in_channels, stride= 1),
                    ResidualBlockEncoder(in_channels=in_channels, out_channels=h_dim, stride= 2))
            )
            in_channels = h_dim

        self.encoder = nn.Sequential(*modules)
        self.fc_mu = nn.Linear(hidden_dims[-1]*self.smallest_size**2, latent_dim)
        self.fc_var = nn.Linear(hidden_dims[-1]*self.smallest_size**2, latent_dim)


        # Build Decoder
        modules = []

        self.decoder_input = nn.Linear(latent_dim, hidden_dims[-1] * self.smallest_size**2)

        hidden_dims.reverse()

        for i in range(len(hidden_dims) - 1):
            modules.append(
                nn.Sequential(
                    ResidualBlockDecoder(hidden_dims[i], hidden_dims[i], stride = 1),
                    ResidualBlockDecoder(hidden_dims[i], hidden_dims[i], stride = 1),
                    ResidualBlockDecoder(hidden_dims[i], hidden_dims[i+1], stride = 2))
            )



        self.decoder = nn.Sequential(*modules)

        self.final_layer = nn.Sequential(
                            nn.BatchNorm2d(hidden_dims[-1]),
                            nn.ConvTranspose2d(hidden_dims[-1],
                                               hidden_dims[-1],
                                               kernel_size=3,
                                               stride=2,
                                               padding=1,
                                               output_padding=1),
                            nn.LeakyReLU(),
                            nn.Conv2d(hidden_dims[-1], out_channels= self.out_channels,
                                      kernel_size= 3, padding= 1),
                            nn.Tanh())

    def encode(self, input: Tensor) -> List[Tensor]:
        """
        Encodes the input by passing through the encoder network
        and returns the latent codes.
        :param input: (Tensor) Input tensor to encoder [N x C x H x W]
        :return: (Tensor) List of latent codes
        """
        result = self.encoder(input)
        result = torch.flatten(result, start_dim=1)

        # Split the result into mu and var components
        # of the latent Gaussian distribution
        mu = self.fc_mu(result)
        log_var = self.fc_var(result)

        return [mu, log_var]

    def decode(self, z: Tensor) -> Tensor:
        """
        Maps the given latent codes
        onto the image space.
        :param z: (Tensor) [B x D]
        :return: (Tensor) [B x C x H x W]
        """
        result = self.decoder_input(z)
        result = result.view(-1, self.nb_last_channels, self.smallest_size, self.smallest_size)
        result = self.decoder(result)
        result = self.final_layer(result)
        return result

    def reparameterize(self, mu: Tensor, logvar: Tensor) -> Tensor:
        """
        Reparameterization trick to sample from N(mu, var) from
        N(0,1).
        :param mu: (Tensor) Mean of the latent Gaussian [B x D]
        :param logvar: (Tensor) Standard deviation of the latent Gaussian [B x D]
        :return: (Tensor) [B x D]
        """
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps * std + mu

    def forward(self, input: Tensor, **kwargs) -> List[Tensor]:
        mu, log_var = self.encode(input)
        z = self.reparameterize(mu, log_var)
        return  [self.decode(z), input, mu, log_var]

    def loss_function(self,
                      *args,
                      **kwargs) -> dict:
        """
        Computes the VAE loss function.
        KL(N(\mu, \sigma), N(0, 1)) = \log \frac{1}{\sigma} + \frac{\sigma^2 + \mu^2}{2} - \frac{1}{2}
        :param args:
        :param kwargs:
        :return:
        """
        recons = args[0]
        input = args[1]
        mu = args[2]
        log_var = args[3]

        kld_weight = kwargs['M_N'] # Account for the minibatch samples from the dataset
        recons_loss =F.mse_loss(recons, input)


        kld_loss = torch.mean(-0.5 * torch.sum(1 + log_var - mu ** 2 - log_var.exp(), dim = 1), dim = 0)

        loss = recons_loss + kld_weight * kld_loss
        return {'loss': loss, 'Reconstruction_Loss':recons_loss, 'KLD':-kld_loss}

    def sample(self,
               num_samples:int,
               current_device: int, **kwargs) -> Tensor:
        """
        Samples from the latent space and return the corresponding
        image space map.
        :param num_samples: (Int) Number of samples
        :param current_device: (Int) Device to run the model
        :return: (Tensor)
        """
        z = torch.randn(num_samples,
                        self.latent_dim)

        z = z.to(current_device)

        samples = self.decode(z)
        return samples

    def generate(self, x: Tensor, **kwargs) -> Tensor:
        """
        Given an input image x, returns the reconstructed image
        :param x: (Tensor) [B x C x H x W]
        :return: (Tensor) [B x C x H x W]
        """

        return self.forward(x)[0]

In [2]:
# Define Trainer

from torchvision.utils import save_image

# Achtung: Momentan wird im Trainer nur Reconstruction Loss verwendet
class Trainer(object):
    def __init__(self, model,
                 optimizer, loss_function,
                 loader_train, loader_val,
                 dtype, device, **in_params):
        """
        :param model: PyTorch model of the neural network

        :param optimizer: PyTorch optimizer

        :param print_every: How often should we print the loss during training
        """
        # Create attributes:
        self.device = device
        self.model = model.to(device=self.device)  # move the model parameters to CPU/GPU
        self.optimizer = optimizer
        self.loss_function = loss_function
        self.loader_train = loader_train
        self.loader_val = loader_val
        self.print_every = in_params["print_every"]
        self.dtype = dtype
        self.batch_size = in_params["batch_size"]
        self.input_size = in_params["input_size"]
        self.path = in_params["path"]


    def train_model(self, epoch):
        """
        - epoch: An integer giving the epoch
        """
        train_loss = 0
        self.model.train()  # put model to training mode
        for t, input in enumerate(self.loader_train):
            
            input = input.to(device=self.device, dtype=self.dtype)  # move to device, e.g. GPU
            
            # do a step in training
            args = self.model(input)
            loss = self.loss_function(*args,**{'M_N':self.batch_size/len(self.loader_train)})['Reconstruction_Loss']
            self.optimizer.zero_grad()
            loss.backward() 
            torch.nn.utils.clip_grad_norm_(self.model.parameters(),1)
            train_loss += loss.item() # accumulate for average loss
            self.optimizer.step()

            # print loss
            if t % self.print_every == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, t * len(args[1]), len(self.loader_train.dataset),
                    100. * t / len(self.loader_train),
                    loss.item() / len(args[1])))
        # print average loss
        print('====> Epoch: {} Average loss: {:.6f}'.format(
              epoch, train_loss / len(self.loader_train.dataset)))

    def test_model(self, epoch):
        self.model.eval() # Put model to evaluation mode
        test_loss = 0.

        with torch.no_grad():
            # During validation, we accumulate these values across the whole dataset and then average at the end:
            for i, input in enumerate(self.loader_val):
                input = input.to(device=self.device, dtype=self.dtype)  # move to device, e.g. GPU
       
                # compute loss and accumulate
                args = self.model(input)
                test_loss += self.loss_function(*args,**{'M_N':self.batch_size/len(self.loader_val)})['Reconstruction_Loss'].item()
                if i == 0:
                    n = min(args[1].size(0), 5)
                    comparison = torch.cat([args[1][:n,:3,:,:],
                                          args[0].view(self.batch_size, self.model.out_channels, self.input_size, self.input_size)[:n,:3,:,:]])
                    save_image(comparison.cpu(),
                             self.path + '/reconstruction_' + str(epoch) + '.png', nrow=n)

        # print average loss
        test_loss /= len(self.loader_val.dataset)
        print('====> Test set loss: {:.6f}'.format(test_loss))
        
    def train_and_test(self, epochs):
        for e in range(1,epochs+1):
            self.train_model(e)
            self.test_model(e)
#            with torch.no_grad():
#                sample = self.model.sample(64,device)
#                save_image(sample.view(64, self.model.out_channels, self.input_size, self.input_size),
#                           self.path + '/sample_' + str(e) + '.png')

In [3]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from dataloader.data_loading import LightFieldDataset

in_params = {"batch_size": 1,
        "epochs": 300,
        "no_cuda": False,
        "seed": 1,
        "print_every": 20,
        "input_size": 512,
        "path": 'results_trial_stacks',
        "in_channels": 9*3,
        "latent_dim": 5300
        }
in_params["cuda"] = not in_params["no_cuda"] and torch.cuda.is_available()
torch.manual_seed(in_params["seed"])

device = torch.device("cuda" if in_params["cuda"] else "cpu")

kwargs = {'num_workers': 0, 'pin_memory': True} if in_params["cuda"] else {}


# transformations of input images before feeding into nn
transformations = None


train_set = LightFieldDataset(sort=['training','additional'], 
                              data_kind = 'hstack', 
                              root_dir = 'full_data', 
                              transform = transformations)
test_set = LightFieldDataset(sort=['test'], 
                              data_kind = 'hstack', 
                              root_dir = 'full_data', 
                              transform = transformations)
train_loader = DataLoader(train_set,
                           batch_size=in_params["batch_size"], 
                           shuffle=True,
                           drop_last=True,
                           num_workers=0,
                           **kwargs
                           )

test_loader = DataLoader(test_set,
                          batch_size=in_params["batch_size"], 
                          shuffle=True,
                          drop_last=True,
                          **kwargs
                          )

print(len(train_loader), len(train_loader.dataset))

180 180


In [4]:
model = VAE(in_channels=in_params['in_channels'],
           in_size=in_params["input_size"],
           latent_dim=in_params['latent_dim'],
           hidden_dims=[9*32, 9*64, 6*128, 3*160])

# Build the optimizer:
params = model.parameters()
learning_rate = 1e-4
optimizer = torch.optim.AdamW(params, lr=learning_rate)


# Build the trainer with the Soresen-Dice loss you implemented:
trainer = Trainer(model, optimizer, model.loss_function,
        train_loader, test_loader, torch.float32, device,**in_params )

# Start training:
trainer.train_and_test(in_params["epochs"])

KeyboardInterrupt: 